In [1]:
from utility import jaccard as jac_np
from utility import point_form as pf_np
from utility import intersect as int_np

from box_utils  import jaccard as jac_torch
from box_utils  import point_form as pf_torch
from box_utils  import intersect  as int_torch 

from CustomDataLoaderTesting import DataAugmentor
import numpy as np
import torch
from pathlib import Path
import collections
from SSD_generate_anchors import generate_ssd_priors

Using TensorFlow backend.


In [2]:
SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

# the SSD orignal specs
specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

priors = generate_ssd_priors(specs)

In [5]:
# root = Path.home()/'data'/'VOCdevkit'/'VOC2007'
root   = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'/'VOC2007'
voc_image_path      = root/'JPEGImages'
voc_annotation_path = root/'Annotations'
voc_trainval_path   = root/'ImageSets'/'Main'/'train.txt'



In [6]:
tester = DataAugmentor()
generator = tester.flow_from_directory(root        = root,
                                       data_file   = voc_trainval_path,
                                       target_size = 300,
                                       batch_size  = 1,
                                       shuffle = True)

TypeError: flow_from_directory() got an unexpected keyword argument 'data_file'

In [ ]:
_ , target = generator[0]

conf_data = target[0][:,0]
loc_data  = target[0][:,1:]

In [ ]:

loc_data_np = pf_np(loc_data)                    # point form numpy 
priors_np   = pf_np(priors).astype(np.float32)   # point form numpy

print(loc_data)
print(loc_data_np)


In [ ]:

loc_data_th = torch.from_numpy(loc_data)
conf_data_th   = torch.from_numpy(conf_data)

priors_th   = torch.from_numpy(priors_np).float()

loc_data_th_pf = pf_torch(loc_data_th) # point from in torch


# print(loc_data_th)
# print(loc_data_th_pf)
print(conf_data)
print(conf_data_th)

In [ ]:
loc_data_th.size()

In [ ]:
iou_np = jac_np(box_a = loc_data_np, box_b = priors_np)
iou_th = jac_torch(box_a = loc_data_th_pf, box_b= priors_th)

In [ ]:
a = (iou_np == iou_th.numpy())

In [ ]:
np.sum(a)

In [ ]:
def encode_np(matched = None, priors = None, variances = [0.1, 0.2]):
    '''
    Encode the variance from the priorbox layers inot the ground truth boxes 
    we have macthed  (based on jaccard overlap) with the prior boxes
    Args:
        matched:  (tensor) coords of ground truth for each prior in point_form 
                   shape = [num_priors, 4]
       priors  : (tensor) priors boxes in center-offset form 
                   shape = [num_priors, 4]
       variance: list(float) Variance of prior boxes

    Returns:
        encoded boxes: (tensor) shape = [num_priors, 4]
    '''
    # dist b/t match center and prior's center
    g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]
    
    # encode variance
    g_cxcy /= (variances[0] * priors[:, 2:])
    
    # match wh / prior wh
    g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]
    g_wh = np.log(g_wh) / variances[1]
    
    return np.concatenate((g_cxcy, g_wh), axis = 1)

def encode_torch(matched, priors, variances):
    """Encode the variances from the priorbox layers into the ground truth boxes
    we have matched (based on jaccard overlap) with the prior boxes.
    Args:
        matched: (tensor) Coords of ground truth for each prior in point-form
            Shape: [num_priors, 4].
        priors: (tensor) Prior boxes in center-offset form
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        encoded boxes (tensor), Shape: [num_priors, 4]
    """

    # dist b/t match center and prior's center
    g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]
    
    # encode variance
    g_cxcy /= (variances[0] * priors[:, 2:])
    
    # match wh / prior wh
    g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]
    g_wh = torch.log(g_wh) / variances[1]
    
    # return target for smooth_l1_loss
    return torch.cat([g_cxcy, g_wh], 1)

In [ ]:
def numpy_argmax(a, axis = 0):
    
    if not axis == 0:
        raise ValueError('to be used only for axis 0')
    
    row, col = a.shape[:2]
    output = []
    for i in range(col):
        x = a[:,i]
        index = np.where(x == x.max(axis = 0))[0][-1] # the last entry in the array
        output.append(index)
        
    return np.array(output)

def index_fill(array, index, axis, value):
    dim = array.shape
    
    if axis == 1:
        for i in range(dim[0]):
            np.put(array[i,:], index, value)
            
    elif axis == 0:
        for i in range(dim[1]):
            np.put(array[:,i], index, value)
            
    return array

def match_np(truths      = None, 
          labels     = None, 
          priors     = None, 
          variance   = None, 
          threshold  = 0.5,
        ):

    
    iou = jac_np(truths, priors)
    
    best_prior_overlap = np.amax(iou, axis=-1).astype(np.float32)
    best_prior_idx     = np.argmax(iou, axis =-1)
    
# #    print(best_prior_overlap.shape)
# #    print(best_prior_idx.shape)

    best_truth_overlap = np.amax(iou, axis=0).astype(np.float32)
    best_truth_idx     = numpy_argmax(iou)
    
    np.put(a= best_truth_overlap, ind = best_prior_idx, v=2)
# #    print(best_truth_overlap.shape)
# #    print(best_truth_idx.shape)

    for j in range(best_prior_idx.shape[0]):
        best_truth_idx[best_prior_idx[j]] = j
    
    matches = truths[best_truth_idx]
    
    conf    = labels[best_truth_idx]
    conf[best_truth_overlap < threshold] = 0
    
    loc     = encode_np(matched=matches, priors=priors, variances=variance)
    
    return loc,conf


def match_torch(threshold = 0.5, 
                truths = None, 
                priors = None, 
                variances = [0.1, 0.2], 
                labels = None):
    
    overlaps = jac_torch(truths, priors)
    
    # (Bipartite Matching)
    # [1,num_objects] best prior for each ground truth
    best_prior_overlap, best_prior_idx = overlaps.max(1, keepdim=True)
#     # [1,num_priors] best ground truth for each prior
    best_truth_overlap, best_truth_idx = overlaps.max(0, keepdim=True)
    
    best_truth_idx.squeeze_(0)
    best_truth_overlap.squeeze_(0)
    best_prior_idx.squeeze_(1)
    best_prior_overlap.squeeze_(1)
    
    best_truth_overlap.index_fill_(0, best_prior_idx, 2)  # ensure best prior
    
    # TODO refactor: index  best_prior_idx with long tensor
    # ensure every gt matches with its prior of max overlap
    for j in range(best_prior_idx.size(0)):
        best_truth_idx[best_prior_idx[j]] = j
        
    matches = truths[best_truth_idx]          # Shape: [num_priors,4]
    
    conf = labels[best_truth_idx]             # Shape: [num_priors]
    conf[best_truth_overlap < threshold] = 0  # label as background
    loc = encode_torch(matches, priors, variances)

    return  loc, conf

In [ ]:
loc_np, conf_np  = match_np(truths=loc_data_np, priors=priors_np, threshold=0.5, variance=[0.1,0.2], labels=conf_data)
loc_th, conf_th  = match_torch(truths=loc_data_th_pf, priors=priors_th, threshold=0.5, labels=conf_data_th)

In [ ]:
c = (loc_np == loc_th.numpy())
np.sum(c)

In [ ]:
def decode_np(loc = None, priors=None, variances = [0.1, 0.2]):
   
    boxes = np.concatenate((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * np.exp(loc[:, 2:] * variances[1])), axis = 1)
    
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes

def decode_torch(loc, priors, variances):
    """Decode locations from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Args:
        loc (tensor): location predictions for loc layers,
            Shape: [num_priors,4]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        decoded bounding box predictions
    """

    boxes = torch.cat((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes

In [ ]:
boxes_np = decode_np(loc= loc_np, priors=priors_np, variances=[0.1, 0.2])
boxes_th = decode_torch(loc=loc_th, priors=priors_th, variances=[0.1, 0.2])

In [ ]:
d = (boxes_np == boxes_th.numpy())
np.sum(d)

In [ ]:
d